In [4]:
import sys
# adding llms to the system path, make sure to configure the file azureopenai in llms folder with end point details
sys.path.append('C:\\proddev\\research\\aiagentssamples\\llms\\')
import azureopenai
from autogen import GroupChat, GroupChatManager
from autogen import AssistantAgent
from autogen import UserProxyAgent


llm_config = {
    "seed": 44,  # for caching the response
    "config_list": azureopenai.config_list,
    "temperature": 1  # for coding it is suggested to be less creative
}

assistant = AssistantAgent(
    "inner-assistant",
    system_message="use yfinance to get the stock data and then use pandas to filter the data.ypfiance and pandas are already installed. Don't install any library.",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

code_interpreter = UserProxyAgent(
    "inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = GroupChat(
    agents=[assistant, code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)


from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent  # noqa: E402

task = "On which days in current year  Microsoft Stock was higher than $370?"

society_of_mind_agent = SocietyOfMindAgent(
    "society_of_mind",
    chat_manager=manager,
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

user_proxy.initiate_chat(society_of_mind_agent, message=task)

user_proxy (to society_of_mind):

On which days in current year  Microsoft Stock was higher than $370?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
society_of_mind (to chat_manager):

On which days in current year  Microsoft Stock was higher than $370?

--------------------------------------------------------------------------------

Next speaker: inner-assistant

inner-assistant (to chat_manager):

To find the days in the current year when Microsoft (MSFT) stock price was higher than $370, you can use the `yfinance` library to download the stock data and `pandas` to filter the data. Here is the process you can follow:

1. Import the necessary libraries.
2. Download the stock data for Microsoft.
3. Filter the data for the current year.
4. Identify the days where the stock price was higher than $370.

Here's the code to do that:

```python
import yfinance as yf
import pandas as pd
from datetime import datetime

# Downloa

ChatResult(chat_id=None, chat_history=[{'content': 'On which days in current year  Microsoft Stock was higher than $370?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'To find the days in the current year when Microsoft (MSFT) stock price was higher than $370, I used the `yfinance` library to download the stock data and filter the dates. Here are the dates when the closing price of Microsoft stock was higher than $370:\n\n### Dates When MSFT Stock Was Higher than $370 in 2024:\n- **January 8, 2024**\n- **January 9, 2024**\n- **January 10, 2024**\n- **January 11, 2024**\n- **January 12, 2024**\n- ... (and several other dates up to September 9, 2024)\n\nThe data provided above is a snippet; the full list includes 169 days spanning from January to September 2024. For a detailed view, the filtered data has been saved to a CSV file named `msft_high_price_days.csv`, containing all relevant information.', 'role': 'user', 'name': 'society_of_mind'}], summary='To find the days in th